In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

#This line will hide code by default when the notebook is exported as HTML
#di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import hail as hl
hl.init(spark_conf={'spark.driver.memory': '10G', 'spark.executor.memory': '10G'}, default_reference='GRCh38') 

In [ ]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()


import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis

from hail.experimental.vcf_combiner.vcf_combiner import combine_gvcfs as cmb

output_notebook() 

In [ ]:
import os
files = os.listdir('/data/data/gvcf/')

In [ ]:
gvcfs = []
for f in files:
    if ((f.find("tbi")) == -1) & (f.find('txt') == -1):
        gvcfs.append('/data/data/gvcf/'+f)
        

In [ ]:
gvcfs.sort()

In [ ]:
len(gvcfs)

In [ ]:
header_try=open('/data/data/gvcf/header.txt').read()

In [ ]:
samples = []

for f in files:
    if ((f.find("tbi")) == -1) & (f.find('txt') == -1):
        samples.append((f.split('.'))[0])

samples.sort()

In [ ]:
hl.experimental.run_combiner(gvcfs, out_file='/data/data/joint/gts-sparse.mt',
                             tmp_path='/tmp', reference_genome='GRCh38', use_genome_default_intervals = True,
                             header = '/data/data/gvcf/header.txt',
                             sample_names = samples)

In [ ]:
sparse = hl.read_matrix_table('/data/data/joint/gts-sparse.mt')

In [ ]:
sparse.count()

In [ ]:
dense = hl.experimental.densify(sparse)

dense.write('/data/data/joint/gts-dense.mt')

In [ ]:
gts = hl.read_matrix_table('/data/data/joint/gts-dense.mt')

In [ ]:
gts.count()

In [ ]:
gts_gt = gts.annotate_entries(GT=hl.experimental.lgt_to_gt(gts.LGT, gts.LA))

In [ ]:
gts_gt.write('/data/data/joint/gts-geno.mt')

In [ ]:
gts_gt = hl.read_matrix_table('/data/data/joint/gts-geno.mt')

In [ ]:
gts_gt.count()

In [ ]:
pheno = hl.import_table('/data/data/external-data/familiespattern.csv')

In [ ]:
to_keep = pheno['sample'].collect()

In [ ]:
gts_gt = gts_gt.filter_cols(hl.literal(to_keep).contains(gts_gt.s))

In [ ]:
gts_gt = gts_gt.filter_rows(hl.agg.any(gts_gt.GT.is_non_ref()))

In [ ]:
gts_gt.write('/data/data/joint/fams-unfiltered.mt')

In [ ]:
gts_gts = hl.read_matrix_table('/data/data/joint/fams-unfiltered.mt')

In [ ]:
gts_gts.count()